# BentoML Example: Time-Series Statistical Model 


**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


Make sure to __use GPU runtime when running this notebook in Google Colab__, you can set it in top menu: `Runtime > Change Runtime Type > Hardware accelerator`.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=statsmodel&ea=statsmodel-timeseries&dt=statsmodel-timeseries)


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install -q bentoml statsmodels==0.10.1

You should consider upgrading via the '/usr/local/anaconda3/envs/dev-py3/bin/python -m pip install --upgrade pip' command.


In [3]:
%%writefile holt.py

# holt.py
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.service.artifacts.common import PickleArtifact
import numpy as np

@env(pip_dependencies=["statsmodels==0.10.1","joblib","numpy"])
@artifacts([PickleArtifact('model')])
class holt(BentoService):
    @api(input=DataframeInput(), batch=True)
    def predict(self, df):

        # Printing the dataframe to cross-cjheck
        print(df.head())

        # Parsing the dataframe values
        weeks=int(df.iat[0,0])
        print(weeks)
        return((self.artifacts.model).forecast(weeks))
  

Overwriting holt.py


The bentoml.api decorator defines a service API, which is the entry point for accessing the prediction service. The DataframeInput here denotes that this service API will convert HTTP JSON request into pandas.DataFrame object before passing it to the user-defined API function code for inference.

The pip_dependencies specify the libraries that you would need in your code. Any library specified here will automatically get added to the requirements.txt folder . 

Here we're using the PickleArtifact. However,  BentoML also provide model artifact for other frameworks such as PytorchModelArtifact, KerasModelArtifact, FastaiModelArtifact, and XgboostModelArtifact etc.

The following code trains a scikit-learn model and bundles the trained model with an Holt instance. The Holt instance is then saved to disk in the BentoML SavedBundle format, which is a versioned file archive that is ready for production models serving deployment,we've considered a shampoo sales data which is available publicly.

In [5]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import joblib
from holt import holt

df=pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv')

#Taking a test-train split of 80 %
train=df[0:int(len(df)*0.8)] 
test=df[int(len(df)*0.8):]

#Pre-processing the  Month  field
train.Timestamp = pd.to_datetime(train.Month,format='%m-%d') 
train.index = train.Timestamp 
test.Timestamp = pd.to_datetime(test.Month,format='%m-%d') 
test.index = test.Timestamp 

#fitting the model based on  optimal parameters
model = ExponentialSmoothing(np.asarray(train['Sales']) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit()

#creating an instance of the holt class
holt_obj = holt()


# Pack the newly trained model artifact
holt_obj.pack('model', model)
saved_path = holt_obj.save()

[2020-09-22 21:28:55,608] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 21:28:56,042] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.


/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  app.launch_new_instance()
/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


[2020-09-22 21:28:57,412] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/setuptools/dist.py:476: UserWarning: Normalizing '0.9.0.pre+6.g4beee0a8.dirty' to '0.9.0rc0+6.g4beee0a8.dirty'
  normalized_version,
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+6.g4beee0a8.dirty/bentoml/_version.py
set BentoML-0.9.0rc0+6.g4beee0a8.dirty/bentoml/_version.py to '0.9.0.pre+6.g4beee0a8.dirty'
[2020-09-22 21:29:01,475] INFO - BentoService bundle 'holt:20200922212857_02D15F' saved to: /Users/bozhaoyu/bentoml/repository/holt/20200922212857_02D15F


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [6]:
!bentoml serve holt:latest

[2020-09-22 21:29:25,793] INFO - Getting latest version holt:20200922212857_02D15F
[2020-09-22 21:29:25,793] INFO - Starting BentoML API server in development mode..
[2020-09-22 21:29:26,096] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 21:29:26,114] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+6.g4beee0a8.dirty
[2020-09-22 21:29:26,424] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
 * Serving Flask app "holt" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):


In [ ]:
!bentoml serve holt:latest --run-with-ngrok

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Run the following `curl` command to send the image to REST API server and get a prediction result:

```
curl -i \
  --header "Content-Type: application/json" \
  --request POST \
  --data '[[2]]' \
  http://localhost:5000/predict
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [7]:
!bentoml containerize holt:latest 

[2020-09-22 21:30:48,575] INFO - Getting latest version holt:20200922212857_02D15F
Found Bento: /Users/bozhaoyu/bentoml/repository/holt/20200922212857_02D15F
[2020-09-22 21:30:48,613] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 21:30:48,627] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+6.g4beee0a8.dirty
Tag not specified, using tag parsed from BentoService: 'holt:20200922212857_02D15F'
Building Docker image holt:20200922212857_02D15F from holt:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/holt/20200922212857_02D15F ['Dockerfile', 'MANIFEST.in', 'README.md', 'bentoml-init.sh', 'bentoml.yml', 'bundled_pip_d

|Requirement already satisfied: python-dateutil<3.0.0,>=2.7.3 in /opt/conda/lib/python3.7/site-packages (from bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (2.8.1)
/Collecting scipy>=0.18
|  Downloading scipy-1.5.2-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)
/Collecting pytz>=2011k
|Requirement already satisfied: six in /opt/conda/lib/python3.7/site-packages (from packaging->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (1.15.0)
\Requirement already satisfied: thriftpy2>=0.4.0 in /opt/conda/lib/python3.7/site-packages (from py-zipkin->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (0.4.11)
-Installing collected packages: pytz, numpy, pandas, patsy, scipy, statsmodels, joblib
|  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
-    Uninstalling numpy-1.19.2:
\      Successfully uninstalled numpy-1.19.2
\Successfully installed joblib-0.14.1 numpy-1.18.4 pandas-0.24.2 patsy-0.5.1 pytz-2020.1 scipy-1.5.2 statsmodels-0.10.1
- ---> 4750311

-Requirement already satisfied, skipping upgrade: certifi in /opt/conda/lib/python3.7/site-packages (from BentoML==0.9.0rc0+6.g4beee0a8.dirty) (2020.6.20)
/Requirement already satisfied, skipping upgrade: async-timeout<4.0,>=3.0 in /opt/conda/lib/python3.7/site-packages (from aiohttp->BentoML==0.9.0rc0+6.g4beee0a8.dirty) (3.0.1)
|  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.9.0rc0+6.g4beee0a8.dirty-py3-none-any.whl size=3058751 sha256=70111534774d574d512f67e82acc43c08e3255b8f20a6268093b77ec2d730339
  Stored in directory: /root/.cache/pip/wheels/04/cb/be/16eecf14d2539252672c27d69a3b88c96604cf94c4c32a2ba7
Successfully built BentoML
|Installing collected packages: BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.9.0rc0
-    Uninstalling BentoML-0.9.0rc0:
\      Successfully uninstalled BentoML-0.9.0rc0
/Successfully installed BentoML-0.9.0rc0+6.g4beee0a8.dirty
/ ---> 860141a5b2

In [8]:
!docker run -p 5000:5000 holt:20200922212857_02D15F

[2020-09-23 04:32:30,915] INFO - Starting BentoML API server in production mode..
[2020-09-23 04:32:31,267] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-23 04:32:31,277] INFO - Running micro batch service on :5000
[2020-09-23 04:32:31 +0000] [11] [INFO] Starting gunicorn 20.0.4
[2020-09-23 04:32:31 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-23 04:32:31 +0000] [11] [INFO] Listening at: http://0.0.0.0:5000 (11)
[2020-09-23 04:32:31 +0000] [1] [INFO] Listening at: http://0.0.0.0:59489 (1)
[2020-09-23 04:32:31 +0000] [1] [INFO] Using worker: sync
[2020-09-23 04:32:31 +0000] [11] [INFO] Using worker: aiohttp.worker.GunicornWebWorker
[2020-09-23 04:32:31 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-23 04:32:31 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-23 04:32:31,325] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, un

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [13]:
from bentoml import load
import pandas as pd

loaded_svc = load(saved_path)

print(loaded_svc.predict(pd.DataFrame(data=[[2]])))

[2020-09-22 21:34:39,426] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+6.g4beee0a8.dirty
[2020-09-22 21:34:39,428] WARNING - Module `holt` already loaded, using existing imported module.
[2020-09-22 21:34:39,432] WARNING - pip package requirement pandas already exist
   0
0  2
2
[487.86681173 415.82743026]


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [10]:
!bentoml run holt:latest predict --input '[[2]]'

[2020-09-22 21:33:21,460] INFO - Getting latest version holt:20200922212857_02D15F
[2020-09-22 21:33:21,499] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 21:33:21,515] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+6.g4beee0a8.dirty
[2020-09-22 21:33:21,831] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
   0
0  2
2
[2020-09-22 21:33:26,555] INFO - {'service_name': 'holt', 'service_version': '20200922212857_02D15F', 'api': 'predict', 'task': {'data': {}, 'task_id': 'f6ab7f63-0dcc-42c1-8bf3-769572ad51dc', 'batch': 1, 'cli_args': ('--input', '[[

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

